Do not use mingw64 to run Jupyter Notebook. Use a CMD or an Anaconda Prompt, and activate the env. Then go to the path which the script is located in.

$ activate r-tensorflow

$ cd E:\deeplearning\Hepatocarcinomes\codes\5x\result_vis\as_prob_map

$ E:

$ jupyter notebook

Realize the same function as 'write_prob_map_tum_little.R' which is commented.

In [1]:
import openslide
import csv
import os
import numpy as np
from PIL import Image

In [2]:
slide_name = 'HMNT0278_bis'

In [3]:
slides_full_name=[]
ndpi_path = 'W:\\HES HCC DL Projet'
for path, dirs, files in os.walk(ndpi_path):
    for name in files:
            if name.endswith('.ndpi'):
                slides_full_name.append(name) 
                print(name)
    break

HMNT0001.ndpi
HMNT0001_bis - 2017-05-31 15.03.09.ndpi
HMNT0003 - 2017-05-29 15.41.33.ndpi
HMNT0069_bis - 2017-07-09 06.02.22.ndpi
HMNT1278 - 2017-07-15 23.39.01.ndpi
HMNT0552 - 2017-06-29 14.35.51.ndpi
HMNT0552_bis - 2017-06-29 15.49.54.ndpi
HMNT0003_bis.ndpi
HMNT0004 - 2017-07-08 21.07.19.ndpi
HMNT0023 - 2017-08-19 00.15.23.ndpi
HMNT0023_bis - 2017-08-19 01.13.00.ndpi
HMNT0024 - 2017-08-18 22.53.34.ndpi
HMNT0024_bis - 2017-08-18 23.44.29.ndpi
HMNT0025 - 2017-08-19 02.59.35.ndpi
HMNT0025_bis - 2017-08-19 03.56.58.ndpi
HMNT0050 - 2017-07-09 02.59.54.ndpi
HMNT0050_bis - 2017-07-09 03.29.35.ndpi
HMNT0053 - 2017-07-15 10.44.18.ndpi
HMNT0053_bis - 2017-07-15 11.25.55.ndpi
HMNT0055 - 2017-07-13 21.59.41.ndpi
HMNT0055_bis - 2017-07-13 22.57.41.ndpi
HMNT0057 - 2017-08-19 01.54.57.ndpi
HMNT0057_bis - 2017-08-19 02.21.27.ndpi
HMNT0060 - 2017-07-08 23.37.28.ndpi
HMNT0060_bis - 2017-07-09 00.29.11.ndpi
HMNT0061 - 2017-05-29 17.48.52.ndpi
HMNT0061_bis - 2017-05-29 18.05.59.ndpi
HMNT0062 - 2017-05-2

In [4]:
def get_slide_info(path):
    for name in slides_full_name:
        if name.startswith(path):
            print(ndpi_path + '\\' + name)
            slide = openslide.OpenSlide(ndpi_path + '\\' + name)
            [m, n] = slide.dimensions
        #     print(m, n)
            umpp = slide.properties[openslide.PROPERTY_NAME_MPP_X]
        #     print(1/float(umpp))
            if umpp == '0.45766590389016021':
                magn = 20
            elif umpp == '0.2288329519450801':
                magn = 40
            else:
                magn = 0
            slide.close()
            return m, n, magn

In [5]:
horizontal, vertical, resu_orin = get_slide_info(slide_name) 
resu_split = 5
field = 512
label_size = 64
n = int(field/label_size)

W:\HES HCC DL Projet\HMNT0242_bis - 2017-05-30 15.53.22.ndpi


In [6]:
num_hor = int(horizontal / (resu_orin / resu_split)) // field + 1
num_ver = int(vertical / (resu_orin / resu_split)) // field + 1

In [7]:
path = 'E:\\deeplearning\\data\\5x\\detector\\train\\' + slide_name + '\\results'

In [8]:
prob = []
with open(path + '\\vgg_dense_bn_prob') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
#             print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            prob.append(row[0])
            prob.append(row[1])
            line_count += 1

In [9]:
label = np.zeros((num_ver*n,num_hor*n)) 
label_field = np.zeros((n, n))
j = 0
left_right = 0
hor_field = np.arange(0, num_hor*n, n)
hor_field_inv = np.arange(num_hor*n-n, -8, -n)
ver_field = np.arange(0, num_ver*n, n)

In [10]:
for cc in ver_field:
    cc_ = cc + n
    if left_right % 2 == 0:
        for rr in hor_field:
            rr_ = rr+n
            for r in range(n): #in fact is process by column
                for c in range(n): #1 to 8
                    label_field[c,r] = prob[j]
                    j = j+2
            label[cc:cc_,rr:rr_] = label_field.copy()
        left_right=left_right+1
    else:
        for rr in hor_field_inv:
            rr_ = rr+n
            for r in range(n): #in fact is process by column
                for c in range(n): #1 to 8
                    label_field[c,r] = prob[j]
                    j = j+2
            label[cc:cc_,rr:rr_] = label_field.copy()
        left_right=left_right+1

In [11]:
formatted = (label * 255 / np.max(label)).astype('uint8')
img = Image.fromarray(formatted)
img.save(path + '\\prob_map_tum_8_subtile.tiff')